# Basic usage of the GraphRAG-SDK to Create a Knowledge Graph and RAG System

The following example demonstrates the basic usage of this SDK to create a GraphRAG using URLs with auto-detected ontology.

In [ ]:
!pip install graphrag_sdk[openai]

In [11]:
from dotenv import load_dotenv
from graphrag_sdk.source import URL
from graphrag_sdk import KnowledgeGraph, Ontology
from graphrag_sdk.models.openai import OpenAiGenerativeModel
from graphrag_sdk.model_config import KnowledgeGraphModelConfig

# Load environment variables
load_dotenv()

# Configuration
# OPENAI_API_KEY = "sk-"# OpenAI API key


True

### Import Source Data from URLs

This example uses rottentomatoes URL files as the source data. We will import these files as `Source` objects.

In [12]:
urls = ["https://www.rottentomatoes.com/m/side_by_side_2012",
"https://www.rottentomatoes.com/m/matrix",
"https://www.rottentomatoes.com/m/matrix_revolutions",
"https://www.rottentomatoes.com/m/matrix_reloaded",
"https://www.rottentomatoes.com/m/speed_1994",
"https://www.rottentomatoes.com/m/john_wick_chapter_4"]

sources = [URL(url) for url in urls]

### Ontology from the Sources

Next, we will utilize an LLM to automatically extract ontology from the data. We will also add `boundaries` to the ontology detection process to ensure the desired ontology is accurately identified.

In [13]:
model = OpenAiGenerativeModel(model_name="gpt-4o")

boundaries = """
    Extract only the most relevant information about all the movies, actors, and directors over the text.
    Avoid creating entities for details that can be expressed as attributes.
"""

ontology = Ontology.from_sources(
    sources=sources,
    boundaries=boundaries,
    model=model,
)

### KG from Sources and Ontology

Create a Knowledge Graph (KG) from the sources.

In [14]:
kg = KnowledgeGraph(
    name="movies",
    model_config=KnowledgeGraphModelConfig.with_model(model),
    ontology=ontology,
)
kg.process_sources(sources)

### Graph RAG

Utilize the `chat_session` method to start a conversations.

In [16]:
chat = kg.chat_session()

print(chat.send_message("Who is the director of the movie The Matrix?"))
print(chat.send_message("How this director connected to Keanu Reeves?"))
print(chat.send_message("Who is the director of the movie Side by Side?"))
print(chat.send_message("Order the directors that you mentioned in all of our conversation by lexical order."))

The directors of the movie "The Matrix" are Lilly Wachowski and Lana Wachowski.
The directors, Lilly Wachowski and Lana Wachowski, are connected to Keanu Reeves through their collaboration on the movies "The Matrix," "The Matrix Reloaded," and "The Matrix Revolutions," in which Keanu Reeves acted.
The director of the movie "Side by Side" is Chris Kenneally.
The directors, ordered lexically, are Chris Kenneally, Lana Wachowski, and Lilly Wachowski.
